# Twitter Data Search Application

In [11]:
# importing required libraries
from fastapi import FastAPI, HTTPException
from pymongo import MongoClient
from psycopg2 import connect, OperationalError
from pydantic import BaseModel
from typing import List

# importing exceptions
from exceptions import *

In [12]:
app = FastAPI()

### Information About the Databases
Fields in the user database:
* 'id'
* 'name'
* 'screen_name'
* 'location'
* 'created_at'
* 'followers_count'
* 'friends_count'      
* 'statuses_count'     (The total number of tweets that the account has posted.)
* 'favourites_count'   (The total number of tweets that the account has liked.)

In [ ]:
# Connect to MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["twitter"]
tweets_collection = db["tweets"]

# Connect to PostgreSQL
try:
    conn = connect(
        dbname="twitter",
        user="postgres",
        host="localhost",
        password="password"
    )
except OperationalError as e:
    print(f"Unable to connect to PostgreSQL: {e}")

cursor = conn.cursor()

### Functionalities of the Search App
The search app supports searches using usernames, screen names, words used in tweets and retweets (key-words) and geographical locations.
Users can query to get user information, like a user's screen name, their bio, follower count, their tweets, etc.
Users can also query on key-words to get tweets related to a certain topics. They can be ordered by date and popularity.

In [ ]:
# Define models
class Tweet(BaseModel):
    id_str: str
    text: str
    created_at: str
    author: str
    retweet_count: int


# Define routes
@app.get("/tweets")
async def search_tweets(query: str, start_date: str = None, end_date: str = None) -> List[Tweet]:
    # Define MongoDB query
    mongo_query = {"$text": {"$search": query}}
    if start_date:
        mongo_query["created_at"] = {"$gte": start_date}
    if end_date:
        mongo_query["created_at"] = {"$lte": end_date}

    # Execute MongoDB query
    results = tweets_collection.find(mongo_query)

    # Convert MongoDB results to Tweet objects
    tweets = [Tweet(**result) for result in results]

    # Get author information from PostgreSQL and add to Tweet objects
    for tweet in tweets:
        cursor.execute(f"SELECT * FROM users WHERE screen_name='{tweet.author}'")
        result = cursor.fetchone()
        if result:
            tweet.author_name = result[1]
            tweet.author_location = result[2]

    return tweets

In [ ]:
def getUserInfo(userid):
    try:
        # search for the userid in the database
        # if found, pull up all the relevant info about the user
        pass
    except:
        raise UserNotFoundError

In [ ]:
def getTweetInfo(tweetid):
    try:
        # search for the tweetid
        # return the tweet, comments, retweet information, etc. if found in the database
        pass
    except:
        raise TweetNotFoundError

In [ ]:
def getKeywordTweets(keywords):
    keyword_list = keywords.split(",")
    for keyword in keywords:
        pass

In [ ]:
def search_twitter(userid = None, tweetid = None, keywords = None):
    """
        This function allows the end-user to query the database based on any of the specified input parameters.
        Multiple input parameters can also be specified for a more comprehensive search, if so desired.
    """
    if userid is not None:
        getUserInfo(userid)

    if tweetid is not None:
        getTweetInfo(tweetid)
    
    if keywords is not None:
        getKeywordTweets(keywords)